In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Hackerearth Challenges/Deeplearning 1')

In [3]:
import pandas as pd
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [ ]:
!unzip data

In [14]:
df = pd.read_csv('./dataset/train.csv')
df.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [15]:
df['Class'].value_counts()

Miscellaneous     2801
Christmas_Tree    1539
Jacket             640
Candle             593
Airplane           535
Snowman            361
Name: Class, dtype: int64

In [16]:
mapp = {}
rev_mapp = {}
curr = 0
for label in df['Class'].unique():
    mapp[label] = curr
    rev_mapp[curr] = label
    curr += 1

In [17]:
temp_data = {}
for index, row in df.iterrows():
    temp_data[row['Image']] = row['Class']

In [18]:
import os
import numpy as np

In [19]:
os.listdir()

['Transfer learning.ipynb',
 'weights.h5',
 'data',
 'BasicCNN.ipynb',
 'out.csv',
 'dataset']

## Training set

In [20]:
xw = 80
xh = 80

In [21]:
path = './dataset/train'
X_train = []
y_train = []

In [ ]:
from tqdm import tqdm
for file in tqdm(os.listdir(path)):
    img = image.load_img(os.path.join(path, file), target_size = (xw, xh))
    x = image.img_to_array(img)
    X_train.append(x)
    y_train.append(mapp[temp_data[file]])

In [23]:
X_train = np.stack(X_train)
y_train = np.stack(y_train)

In [24]:
print(X_train.shape, y_train.shape)

(6469, 80, 80, 3) (6469,)


In [25]:
path = './dataset/test'
test_X = []
file_name = []

for file in os.listdir(path):
    file_name.append(file)
    img = image.load_img(os.path.join(path, file), target_size = (xw, xh))
    x = image.img_to_array(img)
    test_X.append(x)

test_X = np.stack(test_X)
test_X.shape

(3489, 80, 80, 3)

In [26]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if(logs.get('acc') >= .90):
            print("Cancelling Training")
            self.model.stop_training = True
            
callbacks = myCallback()

In [ ]:
!wget https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 -O weights.h5

--2021-01-19 10:44:47--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 74.125.69.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘weights.h5’

weights.h5          100%[===================>]  83.84M  79.9MB/s    in 1.0s    

2021-01-19 10:44:48 (79.9 MB/s) - ‘weights.h5’ saved [87910968/87910968]



In [27]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = 'weights.h5'

pre_trained_model = InceptionV3(input_shape = (xw, xh, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 3, 3, 768)


In [28]:
from tensorflow.keras.optimizers import RMSprop
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(6, activation='softmax')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr = 0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])


In [30]:
history = model.fit(
            X_train, y_train,
            steps_per_epoch = 100,
            epochs = 20)

Epoch 1/20
100/100 [==============================] - 2s 19ms/step - loss: 0.3695 - acc: 0.8678
Epoch 2/20
100/100 [==============================] - 2s 18ms/step - loss: 0.3483 - acc: 0.8797
Epoch 3/20
100/100 [==============================] - 2s 18ms/step - loss: 0.3358 - acc: 0.8848
Epoch 4/20
100/100 [==============================] - 2s 18ms/step - loss: 0.3051 - acc: 0.8952
Epoch 5/20
100/100 [==============================] - 2s 18ms/step - loss: 0.3052 - acc: 0.8974
Epoch 6/20
100/100 [==============================] - 2s 18ms/step - loss: 0.2748 - acc: 0.9052
Epoch 7/20
100/100 [==============================] - 2s 18ms/step - loss: 0.2820 - acc: 0.9025
Epoch 8/20
100/100 [==============================] - 2s 18ms/step - loss: 0.2579 - acc: 0.9127
Epoch 9/20
100/100 [==============================] - 2s 18ms/step - loss: 0.2372 - acc: 0.9178
Epoch 10/20
100/100 [==============================] - 2s 18ms/step - loss: 0.2370 - acc: 0.9215
Epoch 11/20
100/100 [==================

## Prediction

In [31]:
y_hat = model.predict(test_X)

In [32]:
test_y = []

In [33]:
for lis in y_hat:
    index = np.argmax(lis)
    test_y.append(rev_mapp[index])

In [34]:
print(test_y[:5])

['Miscellaneous', 'Miscellaneous', 'Jacket', 'Christmas_Tree', 'Jacket']


In [35]:
res = {}
for i in range(test_X.shape[0]):
    res[file_name[i]] = test_y[i]

In [36]:
result = pd.DataFrame(list(res.items()), columns = ['Image', 'Class'])

In [37]:
result.head()

,Image,Class
0,image8132.jpg,Miscellaneous
1,image1490.jpg,Miscellaneous
2,image3111.jpg,Jacket
3,image664.jpg,Christmas_Tree
4,image5802.jpg,Jacket


In [38]:
print(result.head())

           Image           Class
0  image8132.jpg   Miscellaneous
1  image1490.jpg   Miscellaneous
2  image3111.jpg          Jacket
3   image664.jpg  Christmas_Tree
4  image5802.jpg          Jacket


In [39]:
result.to_csv('out.csv',index = False)

In [40]:
dff = pd.read_csv('out.csv')

In [41]:
dff.head()

,Image,Class
0,image8132.jpg,Miscellaneous
1,image1490.jpg,Miscellaneous
2,image3111.jpg,Jacket
3,image664.jpg,Christmas_Tree
4,image5802.jpg,Jacket
